In [1]:
import pandas as pd
import plotly.express as px
import json
import numpy as np

In [2]:
filename = 'Average_New_Mort_Can.xlsx'
filename_income = 'Canada_median_income.xlsx'


xls=pd.ExcelFile(filename) 
df = pd.read_excel(xls)

xls_income=pd.ExcelFile(filename_income) 
df_income = pd.read_excel(xls_income)

In [3]:
#MOrtgage rates are normally quoted on semianual basis. To find the monthly payments you have to convert to monthly rates
#Converting SemiAnually rate to Quarterly Rate conversion
df['rate_q4'] = ((1+(df['mort_rate_avg_quart']/200))**(1/2)-1)*4

n = 25 #Mortagae term

#Ammortization
df['mort_pay_q4'] = df['Avg_New_Mort']/((1-(1+df['rate_q4']/4)**(-n*4))/(df['rate_q4']/4))


In [4]:
years = df_income['Year'].unique()
df['quart_income'] = 0

provinces = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
       'Newfoundland', 'Nova Scotia', 'Ontario', 'Prince Edward Island',
       'Québec', 'Saskatchewan']

for prov in provinces[:]:
    for y in years:
        income = df_income.loc[(df_income['Prov'] == prov) & (df_income['Year'] == y)]['Median Income'].values[0]  
        df.loc[((df['Provinces'] == prov) & (df['date'].str[:4] == str(y))), "quart_income"] = int(income/4)  

#Calculating Portion of Income Devoted towards paying off mortgages
df['mort_income_port'] = (df['mort_pay_q4']/df['quart_income'])*100

In [5]:
df = df.replace(np.nan,0)
df.loc[((df['date'].str[:4] == '2021') | (df['date'].str[:4] == '2022')), "mort_income_port"] = ''

In [6]:
df.replace([np.inf, -np.inf, ''], np.nan, inplace=True)
df = df.dropna()

In [7]:
df['category'] = ''
#categorizing the number of cases and assign each category to each row
def set_cat(row):
    if row['mort_income_port'] == 0:
        return 'N/A'
    if row['mort_income_port'] > 0 and row['mort_income_port'] < 11:
        return '0 - 10'
    if row['mort_income_port'] > 11 and row['mort_income_port'] < 21:
        return '10 - 20'
    if row['mort_income_port'] > 21 and row['mort_income_port'] < 31:
        return '20 - 30'
    if row['mort_income_port'] > 31 and row['mort_income_port'] < 41:
        return '30 - 40'
    if row['mort_income_port'] > 41:
        return '40 +'

df = df.assign(category=df.apply(set_cat, axis=1))

In [8]:
df.loc[((df['date'].str[:4] == '2018'))].head()

,date,cartodb_id,Provinces,Avg_New_Mort,mort_rate_avg_quart,rate_q4,mort_pay_q4,quart_income,mort_income_port,category
22,2018Q1,4,Alberta,302814,4.223333,0.042013,4906.284779,19100,25.687355,20 - 30
23,2018Q2,4,Alberta,302116,4.313333,0.042903,4940.325801,19100,25.865580,20 - 30
24,2018Q3,4,Alberta,305101,4.426667,0.044024,5047.110330,19100,26.424661,20 - 30
25,2018Q4,4,Alberta,295839,4.483333,0.044585,4922.121909,19100,25.770272,20 - 30
61,2018Q1,6,British Columbia,384794,4.223333,0.042013,6234.549741,15225,40.949424,30 - 40


In [11]:
fig = px.choropleth(df,
                    locations="cartodb_id",
                    geojson="https://raw.githubusercontent.com/Tenzinomics/click_that_hood/main/public/data/canada.geojson",
                    featureidkey="properties.cartodb_id",
                    color="category",
                    color_discrete_map={
                        'N/A': '#F2F2F2',
                        '0 - 10' : '#F0F2DF',
                        '10 - 20' : '#B0D1D9',
                        '20 - 30' : '#8AA6BF',
                        '30 - 40' : '#587AA6',
                        '40 +' : '#35528C'
                    },
                    category_orders={
                      'category' : [
                        'N/A',
                        '0 - 10',
                        '10 - 20',
                        '20 - 30',
                        '30 - 40',
                        '40 +'
                      ]
                    },
                    animation_frame="date",
                    scope='north america',
                    title='<b>Debt service ratio for new mortgage</b>',
                    labels={'mort_income_port' : ' %',
                            'category' : 'Category'},
                    hover_name='Provinces',
                    hover_data={
                        'mort_income_port' : True,
                        'cartodb_id' : False
                    },
                    #height=700,
                    locationmode='geojson-id',
                    )

# Adjust map layout stylings
fig.update_layout(
    showlegend=True,
    legend_title_text='<b>In %</b>',
    font={"size": 18, "color": "#808080", "family" : "calibri"},
    margin={"r":400,"t":60,"l":0,"b":0},
    legend=dict(orientation='v'),
    geo=dict(bgcolor='rgba(0,0,0,0)', lakecolor='#fffcfc')
)

# Adjust map geo options
fig.update_geos(showcountries=False, showcoastlines=False,
                showland=False, fitbounds="locations",
                subunitcolor='white')
fig.show()

In [10]:
#fig.write_html("Geomap of New Mortgage Quarterly Payment as Percentage of Quarterly Median Income.html")